<center>
    <b>RNAge Final Report</b>
    <br>
    <a href="https://github.com/edgeslab/cs418-project-RNAge/blob/master/Final_Report.ipynb">
        https://github.com/edgeslab/cs418-project-RNAge/blob/master/Final_Report.ipynb
    </a>
</center>

In [2]:
#Setting up the environment and importing the necessary data
%run Final_source_code/Init.py
%matplotlib inline

### Introduction
The aim of the project is to:
<ol>
    <li>Characterize and compare the relationship of different tissues with aging.</li>
    <li>Find significant genes participating in aging across different tissues.</li>
</ol>

### Data Collection
<b>Data source:</b>
<br>
The data has been obtained from GTEx portal.
<br>
<u>Data source:</u> https://gtexportal.org/home/datasets
<br>
<u>Data summary:</u> https://gtexportal.org/home/tissueSummaryPage
<br>
This data can be downloaded and placed under 'data' directory.
<br><br>
<b>Data Pre-processing:</b>
<br>
Due to the huge size of the original file, it is split into tissue specific data files for easier processing on local machines.

In [3]:
%run Final_source_code/dataSplit.py